In [ ]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder,OneHotEncoder
import pickle
from tensorflow.keras.models import load_model
import datetime

In [39]:
##Load the trained model
model=load_model('model.h5')
##Load the encoder and scaler
with open('label_encoder_gender.pkl','rb')as file:
    label_encoder_gender=pickle.load(file)
with open('onehot_encoder_geo.pkl','rb')as file:
    onehot_encoder_geo=pickle.load(file)
with open('scaler.pkl','rb')as file:
    scaler=pickle.load(file)
    

In [40]:
input_data = {
    "CreditScore": 619,
    "Geography": "France",
    "Gender": "Female",
    "Age": 42,
    "Tenure": 2,
    "Balance": 0.00,
    "NumOfProducts": 1,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 101348.88
}

In [41]:
#one-hot encode geography
geo = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=  pd.DataFrame(geo,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

e:\Python\Projects\ANN Classification\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [42]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.0,1,1,1,101348.88


In [43]:
#Enocde gender
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,0,42,2,0.0,1,1,1,101348.88


In [44]:
#Concatenate the encoded geography and the input data
input_df=pd.concat([input_df.drop(columns='Geography',axis=1),geo_encoded_df],axis=1)

In [45]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.0,1,1,1,101348.88,1.0,0.0,0.0


In [47]:
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.33880827, -1.09499335,  0.29493847, -1.04241787, -1.21847056,
        -0.91668767,  0.64920267,  0.97481699,  0.01595384,  1.00150113,
        -0.57946723, -0.57638802]])

In [50]:
prediction=model.predict(input_scaled)
print('churn'if prediction[0][0]>0.5 else 'no churn')

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
no churn
